In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium.plugins import MarkerCluster, HeatMap

In [2]:
vci_df = pd.read_csv(os.path.join("data", "vci_county.csv"))

In [3]:
january_vci = {k: v for (k, v) in zip([row.county_code for index, row in vci_df.iterrows()],
                                     [row.vci_jan_17 for index, row in vci_df.iterrows()])}

In [5]:
counties_gdf = gpd.GeoDataFrame.from_file(os.path.join("data", "counties.geojson"))

In [6]:
styles = []

for row in counties_gdf["COUNTY_COD"]:
    try:
        vci_value = january_vci[row]
        if vci_value >= 50:
            styles.append({"fillColor": "#0A9500", "weight": 1, "color": "#000000"})
        elif vci_value >= 35:
            styles.append({"fillColor": "#79C573", "weight": 1, "color": "#000000"})
        elif vci_value >= 21:
            styles.append({"fillColor": "#FFD819", "weight": 1, "color": "#000000"})
        elif vci_value >= 10:
            styles.append({"fillColor": "#FF8B8B", "weight": 1, "color": "#000000"})
        else:
            styles.append({"fillColor": "#FF0000", "weight": 1, "color": "#000000"})
    except KeyError:
        styles.append({"fillColor": "#424242", "weight": 1, "color": "#000000"})

counties_gdf["style"] = styles
counties_gdf.head(2)

,COUNTY_COD,COUNTY_NAM,geometry,style
0,47,NAIROBI,POLYGON ((36.90458259248316 -1.160707895679255...,"{'fillColor': '#424242', 'color': '#000000', '..."
1,1,MOMBASA,POLYGON ((39.68311072648316 -4.060742933679253...,"{'fillColor': '#424242', 'color': '#000000', '..."


In [11]:
vci_map = folium.Map([
    np.mean([row.geometry.centroid.y for index, row in counties_gdf.iterrows()]), 
    np.mean([row.geometry.centroid.x for index, row in counties_gdf.iterrows()])], 
    zoom_start = 6,
    tiles = "Cartodb Positron")

In [11]:
folium.GeoJson(counties_gdf).add_to(vci_map)

vci_map.add_child(MarkerCluster(locations=list(zip([row.geometry.centroid.y for index, row in counties_gdf.iterrows()], 
                                                  [row.geometry.centroid.x  for index, row in counties_gdf.iterrows()])), 
                               popups=[row.COUNTY_NAM for index, row in counties_gdf.iterrows()]))

vci_map.save(os.path.join("html_files", "counties_with_markers.html"))

In [9]:
parks_df = pd.read_csv(os.path.join("data", "parks_reserves.csv"))
parks_df.head(3)

,name,lon,lat
0,SIBILOI NATIONAL PARK,36.349810,4.011805
1,CENTRAL ISLAND NATIONAL PARK,36.038429,3.495894
2,SOUTH ISLAND NATIONAL PARK,36.598594,2.641180


In [10]:
folium.GeoJson(counties_gdf).add_to(vci_map)

vci_map.add_child(MarkerCluster(locations=list(zip([row.lat for index, row in parks_df.iterrows()], 
                                                  [row.lon  for index, row in parks_df.iterrows()])), 
                               popups=[row["name"] for index, row in parks_df.iterrows()]))

HeatMap(list(zip([row.lat for index, row in parks_df.iterrows()], [row.lon  for index, row in parks_df.iterrows()]))).add_to(vci_map)

vci_map.save(os.path.join("html_files", "counties_with_reserves_heatmap.html"))

In [12]:
folium.GeoJson(counties_gdf).add_to(vci_map)

#vci_map.add_child(MarkerCluster(locations=list(zip([row.lat for index, row in parks_df.iterrows()], 
#                                                  [row.lon  for index, row in parks_df.iterrows()])), 
#                               popups=[row["name"] for index, row in parks_df.iterrows()]))

HeatMap(list(zip([row.lat for index, row in parks_df.iterrows()], [row.lon  for index, row in parks_df.iterrows()]))).add_to(vci_map)

vci_map.save(os.path.join("html_files", "counties_with_reserves_heatmap_non_cluster.html"))